In [ ]:
import numpy as np
import qiskit as q
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, QuantumRegister, AncillaRegister
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.cm as cm
import qiskit.circuit.library as qlib
%matplotlib inline
from qiskit import transpile

In [ ]:


M = 16 

dim = 2
dirs = 5
nlat = int(np.ceil(np.log2(M)))
nlinks = int(np.ceil(np.log2(dirs)))

In [ ]:
vort = np.zeros((M,M))
stream = np.zeros((M,M))


Re = 40
U = 1
tau = 3*U*(M-1)/Re + 0.5
wts = (2/6,1/6,1/6,1/6,1/6)
dt=1
cs2 = 1/3  

lambdas = [np.arccos(i) for i in wts]

In [ ]:

def top_half(A):
    def convert(val):
        return val+1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)
def bottom_half(A):
    def convert(val):
        return val-1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)

In [ ]:

def createLambda(oper, isC1 = True):
    ctrl = oper
    if oper >= 8:
        oper -= 8
    s = bin(ctrl)[2:]
    s = '0'*int((4-len(s))) + s
    nlambda = lambdas[oper]
    
    if not isC1:
        nlambda*=-1

    qc = QuantumCircuit(5)
    tmp = QuantumCircuit(1)

    tmp.p(nlambda, 0)
    tmp.x(0)
    tmp.p(nlambda, 0)
    tmp.x(0)
    
    gate = tmp.to_gate(label = "Lambda"+str(ctrl)).control(4,ctrl_state = s)
    
    cbits = [i for i in range(1,5)]
    cbits.append(0)
    qc.append(gate,cbits)
    return qc

In [ ]:

def rshift(n):
    circ = QuantumCircuit(n)
    for i in range(n):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1,n)), i)
    return circ

def lshift(n):
    circ = QuantumCircuit(n)
    for i in reversed(range(n)):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1,n)), i)
    return circ

In [ ]:
def streamCirc():
    q = QuantumRegister(12,'q')
    a = AncillaRegister(1,'a')

    setup = QuantumCircuit(q)
    setup.add_register(a)

    h1 = qlib.HGate().control(1,ctrl_state = 0)

    cb = [a,0]
    cb.extend([i for i in range(8, 12)])
    setup.h(a)
    for i in range(dirs):
        setup.append(createLambda(i,True).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 0),cb)
    for i in range(8,13):
        setup.append(createLambda(i,True).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 0),cb)
    for i in range(dirs):
        setup.append(createLambda(i,False).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 1),cb)
    for i in range(8,13):
        setup.append(createLambda(i,False).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 1),cb)
    setup.h(q[11])
    setup.h(a)
    
    setup.barrier()
    R1 = rshift(4).to_gate(label = "R").control(4,ctrl_state = '0001')
    L1 = lshift(4).to_gate(label = "L").control(4,ctrl_state = '0010')
    R2 = rshift(4).to_gate(label = "R").control(4,ctrl_state = '0011')
    L2 = lshift(4).to_gate(label = "L").control(4,ctrl_state = '0100')
    setup.append(R1,[8,9,10,a[0],3,2,1,0])
    setup.append(L1,[8,9,10,a[0],3,2,1,0])
    setup.append(R2,[8,9,10,a[0],7,6,5,4])
    setup.append(L2,[8,9,10,a[0],7,6,5,4])
    setup.barrier()


    setup.h(8)
    setup.h(9)
    setup.h(10)
    
    return setup
streamCirc().draw()

In [ ]:

def uv(stream):
    
    u =      (stream[2:, 1:-1] - stream[:-2, 1:-1]) / 2
    v = -1 * (stream[1:-1, 2:] - stream[1:-1, :-2]) / 2

    u = np.pad(u, ((1, 1), (1, 1)), mode='edge') 
    v = np.pad(v, ((1, 1), (1, 1)), mode='edge')

    return u, v
    #vor = np.zeros((M, M))

def collision_f(u,v): 
    ex = (0,1,-1,0,0) 
    ey = (0,0,0,1,-1) 

    dudx = (u[1:-1,2:] - u[1:-1, :-2])/2
    dudy = (u[2:,1:-1] - u[:-2, 1:-1])/2
    dvdx = (v[1:-1,2:] - v[1:-1, :-2])/2
    dvdy = (v[2:,1:-1] - v[:-2, 1:-1])/2

    dudx = np.pad(dudx , ((1, 1), (1, 1)), mode='edge')  
    dudy = np.pad(dudy, ((1, 1), (1, 1)), mode='edge')  
    dvdx = np.pad(dvdx, ((1, 1), (1, 1)), mode='edge')  
    dvdy = np.pad(dvdy, ((1, 1), (1, 1)), mode='edge')  

    vorts = np.zeros((5, M, M))
    feq = np.zeros((5, M, M))  

    for link in range(5):     
        feq[link] = wts[link]*(1+(ex[link]*u+ey[link]*v)/cs2)
        vorts[link] = feq[link]*(1 - (tau-1)*dt /cs2*((ex[link]-u)*((ex[link]-u)*dudx+(ey[link]-v)*dudy)+(ey[link]-v)*((ex[link]-u)*dvdx+(ey[link]-v)*dvdy)))  

    return vorts.flatten()



In [ ]:
def vortCirc(stream):
    q = QuantumRegister(12,'q')

    setup = QuantumCircuit(q)
    
    h1 = qlib.HGate().control(1,ctrl_state = 0)
    h3 = qlib.HGate().control(2,ctrl_state = '00')
    
    u, v = uv(stream)

    A_diag = collision_f(u, v)
    
    zeros = np.zeros(M*M)
    A_diag = np.concatenate((A_diag, zeros, zeros, zeros))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    setup.h(11)
    
    Col1 = QuantumCircuit(QuantumRegister(11))
    Col1.diagonal(list(B1_diag),qubit = [0,1,2,3,4,5,6,7,8,9,10])
    Col1 = Col1.to_gate(label='c1')
    
    Col2 = QuantumCircuit(QuantumRegister(11))
    Col2.diagonal(list(B2_diag),qubit = [0,1,2,3,4,5,6,7,8,9,10])
    Col2 = Col2.to_gate(label='c2')
    
    setup.append(Col1.control(1,ctrl_state = '0'),[11,0,1,2,3,4,5,6,7,8,9,10])
    setup.append(Col2.control(1,ctrl_state = '1'),[11,0,1,2,3,4,5,6,7,8,9,10])

    setup.h(11)

    setup.barrier()
    R1 = rshift(4).to_gate(label = "R").control(nlinks,ctrl_state = '001')
    L1 = lshift(4).to_gate(label = "L").control(nlinks,ctrl_state = '010')
    R2 = rshift(4).to_gate(label = "R").control(nlinks,ctrl_state = '011')
    L2 = lshift(4).to_gate(label = "L").control(nlinks,ctrl_state = '100')
    setup.append(R1,[8,9,10,3,2,1,0])
    setup.append(L1,[8,9,10,3,2,1,0])
    setup.append(R2,[8,9,10,7,6,5,4])
    setup.append(L2,[8,9,10,7,6,5,4])
    setup.barrier()


    setup.h(8)
    setup.h(9)
    setup.h(10)

    return setup

vortCirc(stream).draw()

In [ ]:
def calcBounds(stream):
    arr = np.zeros((M,M))
    arr[-1]  = -2*stream[-2]-2*U
    arr[:,0] = -2*stream[:,1]
    arr[:,-1]= -2*stream[:,-2]
    arr[0]   = -2*stream[1]
    return arr

In [ ]:
def Vtimestep(vort, stream):
    bounds = calcBounds(stream)
    zeros = np.zeros((M,M))
    vort = np.concatenate((vort,vort,vort,vort,vort,zeros,zeros,zeros)).flatten()
    vortSV = Statevector(vort).expand([1,0]).evolve(vortCirc(stream))
    vortAr = np.reshape(np.array(vortSV)[:M*M],(M,M))
    vortAr = np.real(vortAr)*2**(3/2)
    vortAr[0] = bounds[0]
    vortAr[-1] = bounds[-1]
    vortAr[:,0] = bounds[:,0]
    vortAr[:,-1] = bounds[:,-1]
    return np.reshape(vortAr,(M,M))

def Stimestep(stream, source):
    zeros = np.zeros((M,M))
    stream = np.concatenate((stream,stream,stream,stream,stream,zeros,zeros,zeros,source,source,source,source,source,zeros,zeros,zeros)).flatten()
    streamSV = Statevector(stream).expand([1,0]).evolve(streamCirc())
    streamAr = np.reshape(np.array(streamSV)[:M*M],(M,M))
    streamAr = np.real(streamAr)*2**(4/2)
    streamAr[0] = 0
    streamAr[-1] = 0
    streamAr[:,0] = 0
    streamAr[:,-1] = 0
    return np.reshape(streamAr,(M,M))

In [ ]:
allstream = []
allvort = []
allstream.append(stream)
allvort.append(vort)

In [ ]:
steps = 300
for i in range(steps):
    
    stream, vort = allstream[-1], allvort[-1]
    stream = Stimestep(stream,vort)
    vort = Vtimestep(vort, stream)

    allstream.append(stream)
    allvort.append(vort)
    np.savetxt('twoCirc16-40/stream'+str(i+1)+'.csv', allstream[-1], delimiter=',')
    np.savetxt('twoCirc16-40/vort'+str(i+1)+'.csv', allvort[-1], delimiter=',')
    u =  np.gradient(stream, axis=0)  
    v = -np.gradient(stream, axis=1) 
    np.savetxt(f'twoCirc16-40/u_gradient_t{i+1}.csv', u, delimiter=',')
    np.savetxt(f'twoCirc16-40/v_gradient_t{i+1}.csv', v, delimiter=',')
    
    if (i + 1) % 10 == 0:
        fig, ax = plt.subplots()
        CS = ax.contour(np.arange(M), np.arange(M), stream, levels=25)
        ax.clabel(CS, inline=True, fontsize=10)
        ax.set_title(f't={i+1}')
        ax.set_aspect('equal', adjustable='box')
        
        X, Y = np.meshgrid(np.arange(M), np.arange(M))
        
        CS = ax.contour(X, Y, stream, levels=25)
        ax.clabel(CS, inline=True, fontsize=10)
        ax.set_title(f't={i+1}')
        ax.set_aspect('equal', adjustable='box')
        
        plt.savefig(f'twoCirc16-40/contour_t{i+1}.png', bbox_inches='tight')
        plt.close(fig)  
        